# Purpose of the Code

The goal of this notebook is to extract EMG signals recorded using Trigno system, process the raw data, extract meaningful features, and store the resulting information in a structured DataFrame format.


## Environment Setup

In [1]:
# pip install libemg  # Required only if you have never installed the libemg package
# pip install --upgrade paramiko cryptography  # Needed during first-time installation of libemg to resolve dependencies

In [2]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append('../../lib')
from import_emg_file import import_data_from_trigno
from trigno_utils import trigno_dataframe_edit, trigno_extract_muscle_emg
from emg_utils import plot_emg_signal, emg_filters, extract_emg_windows, extract_emg_features
from features_utils import combine_multiple_features_lists, detect_segments, assign_emg_labels

In [15]:
# Parameters
intensity = "light"  # Intensity of the task: "light", "medium", or "heavy"
muscle_name = "biceps"  # Target muscle: "biceps", "triceps", "forearm1", or "forearm2"
w_d = 0.2  # Window duration in seconds for feature extraction
ov = 0.5  # Overlap percentage between windows (e.g., 0.5 = 50%)
n_lifts = 3  # Number of lifts

# Feature lists
features_list_norm = ['MAV', 'WL', 'RMS', 'VAR', 'IAV', 'KURT', 'SKEW', 'AR4', 'LD', 'SAMPEN', 'MPK']  # Features on normalized signal
features_list_filt = ['ZC', 'SSC', 'WAMP']  # Features on filtered signal
features_list_freq = ['MNF', 'MNP', 'SM', 'ISD']  # Frequency-domain features

# Paths for input and output
input_folder = f'../../data/acquisitions/trigno/05_14/{intensity}/'  # Input folder with raw data
output_folder = '../../data/extracted_features/trigno/S01_05_14/'  # Output folder to save extracted features
output_filename = f"features_{intensity}_{muscle_name}.csv"  # Name of the output CSV file

# Loading Data Files

In [4]:
dataframes = import_data_from_trigno(input_folder)  # Import raw data from Trigno folder
dataframes = [trigno_dataframe_edit(df) for df in dataframes]  # Format each imported dataframe

In [5]:
muscle_emg_raw, emg_time = trigno_extract_muscle_emg(dataframes, muscle_name)  # Extract EMG signals and timestamps for the specified muscle
# plot_emg_signal(muscle_emg_raw[0], emg_time[0], title=f"{muscle_name} EMG Raw Signal #1")  #Plot raw EMG signal

# EMG Signal Analysis

## Filtering, Rectification, Smoothing and Normalization

In [6]:
fs_list = []  # Initialize list to store sampling frequencies
for time in emg_time:
    fs = 1.0 / np.mean(np.diff(time))  # Calculate sampling frequency from time intervals
    fs_list.append(fs)  

In [7]:
muscle_emg_filtered, _, _, muscle_emg_normalized = emg_filters(muscle_emg_raw, emg_time, fs_list, muscle_name)  # Filter EMG signals
# plot_emg_signal(muscle_emg_normalized[0], emg_time[0], title=f"{muscle_name} EMG Normalized Signal #1", ylabel="%MVC")  # Plot normalized EMG signal

## Windowing and Features Extraction

In [8]:
# Windowing
normalized_win, filtered_win, time_win = extract_emg_windows(
    muscle_emg_normalized, muscle_emg_filtered, emg_time, fs_list,
    window_duration=w_d, overlap=ov)  # Extract windows with specified duration and overlap

In [9]:
# Features Extraction
time_features_norm = extract_emg_features(normalized_win, features_list_norm)  # Extract time-domain features from normalized EMG windows
time_features_filt = extract_emg_features(filtered_win, features_list_filt)  # Extract time-domain features from filtered EMG windows
freq_features_norm = extract_emg_features(normalized_win, features_list_freq)  # Extract frequency-domain features from normalized EMG windows

In [10]:
dfs = combine_multiple_features_lists(time_features_norm, time_features_filt, freq_features_norm)  # Combine all feature lists into a single dataframe

## Label Assignment

In [16]:
all_bkps_list = detect_segments(muscle_emg_normalized, emg_time, intensity, n_lifts, plot=False)  # Detect lifts segments

In [13]:
dfs['label'] = assign_emg_labels(all_bkps_list, normalized_win, fs_list, w_d, ov, intensity)  # Assign activity labels

# Features Export

In [14]:
output_path = os.path.join(output_folder, output_filename)  # Create full path for output CSV file
dfs.to_csv(output_path, index=False)  # Save dataframe with features and labels to CSV